In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
credits=pd.read_csv("/content/drive/MyDrive/credits.csv")
movies=pd.read_csv("/content/drive/MyDrive/movies.csv" )

In [4]:
movies=movies.merge(credits, on='title')

In [5]:
movies.shape

(4808, 23)

In [6]:
#genres,keywords,id,title,overview,cast,crew
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.dropna(inplace=True)

In [9]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.duplicated().sum()

0

In [11]:
#genres
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [12]:
import ast
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [13]:
movies['genres']=movies['genres'].apply(convert)

In [14]:
movies['keywords']=movies['keywords'].apply(convert)

In [15]:
def convert3(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter !=3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [16]:
movies['cast']=movies['cast'].apply(convert)

In [17]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
      break
  return L

In [18]:
movies['crew']=movies['crew'].apply(fetch_director)

In [19]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [20]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [21]:
movies['tags']=movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [22]:
new_df=movies[['movie_id','title','tags']]

In [23]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-23-53a061086f2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [24]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [25]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

<ipython-input-25-242d679aab18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [26]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [27]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [28]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [29]:
new_df['tags']=new_df['tags'].apply(stem)

<ipython-input-29-402dd7cb6b10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [30]:
import nltk

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [32]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
new_df['tags']

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4803    el mariachi just want to play hi guitar and ca...
4804    a newlyw couple' honeymoon is upend by the arr...
4805    "signed, sealed, delivered" introduc a dedic q...
4806    when ambiti new york attorney sam is sent to s...
4807    ever sinc the second grade when he first saw h...
Name: tags, Length: 4805, dtype: object

In [35]:
similarity = cosine_similarity(vectors)

In [36]:
similarity

array([[1.        , 0.06745406, 0.07733089, ..., 0.04357102, 0.        ,
        0.        ],
       [0.06745406, 1.        , 0.07476672, ..., 0.02106314, 0.        ,
        0.02222771],
       [0.07733089, 0.07476672, 1.        , ..., 0.02414726, 0.        ,
        0.        ],
       ...,
       [0.04357102, 0.02106314, 0.02414726, ..., 1.        , 0.04307305,
        0.04307305],
       [0.        , 0.        , 0.        , ..., 0.04307305, 1.        ,
        0.09090909],
       [0.        , 0.02222771, 0.        , ..., 0.04307305, 0.09090909,
        1.        ]])

In [37]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [38]:
recommend('The Shawshank Redemption')

Penitentiary
Prison
Rosewater
Flying By
1982


In [39]:
import pickle

In [40]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [41]:
pickle.dump(similarity,open('similarity1.pkl','wb'))

In [42]:
import os

path = '/content/'
files = os.listdir(path)
print(files)


['.config', 'drive', 'similarity1.pkl', 'movie_dict.pkl', 'sample_data']


In [43]:
import os

file_path = '/content/movie_dict.pkl'
if os.path.exists(file_path):
    print("File exists")
else:
    print("File does not exist")


File exists


In [44]:
import os

file_path = '/content/similarity1.pkl'
if os.path.exists(file_path):
    print("File exists")
else:
    print("File does not exist")


File exists


In [45]:
import shutil

source = '/content/movie_dict.pkl'  # Source path of the 'movies.pkl' file
destination = '/content/drive/MyDrive/movie_dict.pkl'  # Destination path in Google Drive

shutil.move(source, destination)


'/content/drive/MyDrive/movie_dict.pkl'

In [46]:
source = '/content/similarity1.pkl'  # Source path of the 'movies.pkl' file
destination = '/content/drive/MyDrive/similarity1.pkl'  # Destination path in Google Drive

shutil.move(source, destination)

'/content/drive/MyDrive/similarity1.pkl'